In [1]:
import pandas as pd
import numpy as np
import os
import time


In [11]:
data = pd.read_csv('./data/predict_raw/predict_종관.csv', engine='python', encoding='ANSI')
data

,지점,지점명,일시,기온(°C),기온 QC플래그,강수량(mm),강수량 QC플래그,풍속(m/s),풍속 QC플래그,풍향(16방위),...,최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),지면온도 QC플래그,5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,98,동두천,2021-03-19 00:00,10.8,NaN,NaN,NaN,1.0,NaN,20,...,NaN,5000.0,NaN,NaN,8.6,NaN,NaN,NaN,NaN,NaN
1,98,동두천,2021-03-19 01:00,10.6,NaN,NaN,NaN,1.1,NaN,50,...,27.0,5000.0,NaN,NaN,8.6,NaN,NaN,NaN,NaN,NaN
2,98,동두천,2021-03-19 02:00,10.3,NaN,NaN,NaN,0.8,NaN,360,...,27.0,5199.0,NaN,NaN,8.3,NaN,NaN,NaN,NaN,NaN
3,98,동두천,2021-03-19 03:00,10.3,NaN,NaN,NaN,0.1,NaN,0,...,27.0,5299.0,NaN,NaN,8.1,NaN,NaN,NaN,NaN,NaN
4,98,동두천,2021-03-19 04:00,10.1,NaN,NaN,NaN,1.2,NaN,360,...,30.0,5013.0,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4603,203,이천,2021-04-19 19:00,16.8,NaN,NaN,NaN,1.7,NaN,290,...,NaN,1242.0,NaN,NaN,15.4,NaN,NaN,NaN,NaN,NaN
4604,203,이천,2021-04-19 20:00,15.2,NaN,NaN,NaN,1.8,NaN,270,...,NaN,1175.0,NaN,NaN,13.7,NaN,NaN,NaN,NaN,NaN
4605,203,이천,2021-04-19 21:00,13.3,NaN,NaN,NaN,1.6,NaN,270,...,NaN,1079.0,NaN,NaN,12.4,NaN,NaN,NaN,NaN,NaN
4606,203,이천,2021-04-19 22:00,12.5,NaN,NaN,NaN,1.8,NaN,270,...,NaN,1118.0,NaN,NaN,11.5,NaN,NaN,NaN,NaN,NaN


In [12]:
## 원하는 지역

location = ['동두천', '수원', '양평', '이천', '파주', '서울']

In [13]:
# 경기도, 서울 지점만 추출

pre_data = data.loc[np.isin(data['지점명'], location)].reset_index(drop=True)

In [14]:
# 경기도 안의 지점은 모두 경기도로 통일

pre_data.loc[pre_data['지점명']!='서울', '지점명'] = '경기도'

In [15]:
# 지점 index 삭제

pre_data.drop('지점', axis=1, inplace=True)

In [16]:
# 시간 부분 삭제

pre_data['날짜'] = pre_data['일시'].astype('str').str[:-6]
pre_data['시간'] = pre_data['일시'].astype('str').str[11:13]

In [17]:
# 지점, 일시 별로 평균값 산출

final_data = pre_data.groupby(['지점명', '날짜','시간']).mean().reset_index()

In [18]:
# 3자리수 반올림

weather = np.round(final_data, 3)

In [19]:
# 데이터 확인

weather.columns

Index(['지점명', '날짜', '시간', '기온(°C)', '기온 QC플래그', '강수량(mm)', '강수량 QC플래그',
       '풍속(m/s)', '풍속 QC플래그', '풍향(16방위)', '풍향 QC플래그', '습도(%)', '습도 QC플래그',
       '증기압(hPa)', '이슬점온도(°C)', '현지기압(hPa)', '현지기압 QC플래그', '해면기압(hPa)',
       '해면기압 QC플래그', '일조(hr)', '일조 QC플래그', '일사(MJ/m2)', '적설(cm)', '3시간신적설(cm)',
       '전운량(10분위)', '중하층운량(10분위)', '최저운고(100m )', '시정(10m)', '지면상태(지면상태코드)',
       '현상번호(국내식)', '지면온도(°C)', '지면온도 QC플래그', '5cm 지중온도(°C)', '10cm 지중온도(°C)',
       '20cm 지중온도(°C)', '30cm 지중온도(°C)'],
      dtype='object')

In [20]:
weather

,지점명,날짜,시간,기온(°C),기온 QC플래그,강수량(mm),강수량 QC플래그,풍속(m/s),풍속 QC플래그,풍향(16방위),...,최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),지면온도 QC플래그,5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,경기도,2021-03-19,00,11.04,NaN,NaN,NaN,1.12,NaN,136.0,...,34.00,2230.8,NaN,NaN,8.96,NaN,NaN,NaN,NaN,NaN
1,경기도,2021-03-19,01,11.02,NaN,NaN,NaN,1.60,NaN,92.0,...,33.60,2353.4,NaN,40.0,8.78,NaN,NaN,NaN,NaN,NaN
2,경기도,2021-03-19,02,10.52,NaN,NaN,NaN,1.04,NaN,150.0,...,32.75,2311.2,NaN,40.0,8.52,NaN,NaN,NaN,NaN,NaN
3,경기도,2021-03-19,03,10.36,NaN,NaN,NaN,1.10,NaN,60.0,...,31.60,2301.6,NaN,40.0,8.32,NaN,NaN,NaN,NaN,NaN
4,경기도,2021-03-19,04,10.06,NaN,NaN,NaN,1.46,NaN,132.0,...,33.20,2361.0,NaN,40.0,8.20,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,서울,2021-04-19,19,14.60,NaN,NaN,NaN,2.90,NaN,180.0,...,NaN,2000.0,NaN,NaN,14.10,NaN,17.4,16.2,14.7,13.3
1532,서울,2021-04-19,20,13.20,NaN,NaN,NaN,3.70,NaN,180.0,...,NaN,2000.0,NaN,NaN,12.30,NaN,16.6,15.8,14.8,13.5
1533,서울,2021-04-19,21,12.20,NaN,NaN,NaN,3.30,NaN,200.0,...,NaN,2000.0,NaN,NaN,11.30,NaN,15.9,15.4,14.8,13.7
1534,서울,2021-04-19,22,11.60,NaN,NaN,NaN,2.50,NaN,230.0,...,NaN,2000.0,NaN,NaN,10.60,NaN,15.3,14.9,14.7,13.9


In [21]:
final_weather = weather[['지점명', '날짜', '시간', '기온(°C)', '강수량(mm)','풍속(m/s)','습도(%)', '전운량(10분위)','적설(cm)']]

final_weather.columns = ['광역시도명', '날짜', '시간대별 시간', '기온', '강수량', '풍속', '습도', '운량', '적설']

In [22]:
# csv 파일 내보내기

final_weather.to_csv('data/predict/predict_기상데이터.csv', encoding='utf-8')